In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [5]:
import zipfile
import os


zip_file_path = '/content/drive/MyDrive/paper1/level3_gans_1.zip'

extracted_folder_path = '/content/drive/MyDrive/paper1/'


os.makedirs(extracted_folder_path, exist_ok=True)


with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)


extracted_contents = os.listdir(extracted_folder_path)
print(f"Contents of the extraction folder: {extracted_contents}")


Contents of the extraction folder: ['final_folder.zip', 'Level2.zip', 'level3_diffusion.zip', 'final_folder', '__MACOSX', 'Level2', 'level3_diffusion', 'fine_tuned_resnet18_level1_dalle+real.pth', 'level3_gans.zip', 'level3_gans', 'level3_gans_1.zip', 'level3_gans_1']


In [6]:
train_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/paper1/level3_gans_1', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)

In [7]:
resnet18 = models.resnet18(pretrained=True)

num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, 5)

resnet18 = resnet18.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.parameters(), lr=0.005, momentum=0.8)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 94.0MB/s]


In [8]:
num_epochs = 4
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)


        optimizer.zero_grad()


        outputs = resnet18(inputs)
        loss = criterion(outputs, labels)


        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

print("Training finished")

Epoch 1/4, Loss: 1.11888490319252
Epoch 2/4, Loss: 0.3805174112319946
Epoch 3/4, Loss: 0.23819465190172195
Epoch 4/4, Loss: 0.17525964081287385
Training finished


In [9]:
torch.save(resnet18.state_dict(), '/content/drive/MyDrive/paper1/fine_tuned_resnet18_level3_gans.pth')

In [10]:
import torch
from torchvision import transforms
from PIL import Image


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


model = models.resnet18()
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)  
model.load_state_dict(torch.load('/content/drive/MyDrive/paper1/fine_tuned_resnet18_level3_gans.pth'))
model.eval()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)




In [11]:
import zipfile
import os


zip_file_path = '/content/drive/MyDrive/paper1/level3_gans_test.zip'


extracted_folder_path = '/content/drive/MyDrive/paper1/' 


os.makedirs(extracted_folder_path, exist_ok=True)


with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)


extracted_contents = os.listdir(extracted_folder_path)
print(f"Contents of the extraction folder: {extracted_contents}")

Contents of the extraction folder: ['final_folder.zip', 'Level2.zip', 'level3_diffusion.zip', 'final_folder', '__MACOSX', 'Level2', 'level3_diffusion', 'fine_tuned_resnet18_level1_dalle+real.pth', 'level3_gans.zip', 'level3_gans', 'level3_gans_1.zip', 'level3_gans_1', 'fine_tuned_resnet18_level2.pth', 'fine_tuned_resnet18_level3_diffusion.pth', 'fine_tuned_resnet18_level3_gans.pth', 'level3_gans_test.zip', 'level3_diffusion_test.zip', 'level3_diffusion_test', 'level3_gans_test']


In [13]:

image_path = '/content/drive/MyDrive/paper1/level3_gans_1/biggan/00126057.png'
image = Image.open(image_path).convert('RGB')
input_tensor = transform(image)
input_batch = input_tensor.unsqueeze(0).to(device) 


with torch.no_grad():
    output = model(input_batch)


probabilities = torch.nn.functional.softmax(output[0], dim=0)
predicted_class = torch.argmax(probabilities).item()


print(f"Predicted class: {predicted_class}")
print(f"Class probabilities: {probabilities}")

Predicted class: 0
Class probabilities: tensor([0.8790, 0.0912, 0.0133, 0.0089, 0.0077])


In [14]:
import os
from PIL import Image


folder_path = '/content/drive/MyDrive/paper1/level3_gans_test/'


l = ["biggan","gaugan","stargan","stylegan","whichfaceisreal"]
count3 = 0
for i in l:
  files = os.listdir(folder_path+i)


  image_files = [file for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
  count = 1
  count2 = 0

  for image_file in image_files:

    image_path = os.path.join(folder_path+i, image_file)

    try:
        image = Image.open(image_path).convert('RGB')
        input_tensor = transform(image)
        input_batch = input_tensor.unsqueeze(0).to(device)

        with torch.no_grad():
          output = model(input_batch)

        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        predicted_class = torch.argmax(probabilities).item()


        print(f"Predicted class: {predicted_class}")
        print(f"Class probabilities: {probabilities}")

        if(predicted_class == count3):
          print("yes")
          count2 += 1
        image.close()
        print(count)
        count += 1
        print(f"Opened image: {image_path}")
    except Exception as e:
        print(f"Error opening image {image_path}: {e}")
  print(f"the percentage of predicting {i} images is : {(count2*100)/count}")
  count3 += 1

Predicted class: 0
Class probabilities: tensor([0.7886, 0.2007, 0.0017, 0.0067, 0.0022])
yes
1
Opened image: /content/drive/MyDrive/paper1/level3_gans_test/biggan/00001508.png
Predicted class: 0
Class probabilities: tensor([0.8568, 0.0272, 0.0088, 0.0843, 0.0230])
yes
2
Opened image: /content/drive/MyDrive/paper1/level3_gans_test/biggan/00012206.png
Predicted class: 0
Class probabilities: tensor([0.6434, 0.2926, 0.0133, 0.0468, 0.0039])
yes
3
Opened image: /content/drive/MyDrive/paper1/level3_gans_test/biggan/00006071.png
Predicted class: 0
Class probabilities: tensor([0.9470, 0.0232, 0.0038, 0.0246, 0.0014])
yes
4
Opened image: /content/drive/MyDrive/paper1/level3_gans_test/biggan/00018475.png
Predicted class: 0
Class probabilities: tensor([0.6941, 0.2617, 0.0106, 0.0093, 0.0243])
yes
5
Opened image: /content/drive/MyDrive/paper1/level3_gans_test/biggan/00013246.png
Predicted class: 0
Class probabilities: tensor([0.6352, 0.0917, 0.0141, 0.2552, 0.0039])
yes
6
Opened image: /content/dr

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
